In [1]:
# загружаю необходимые библиотеки
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

from sklearn.feature_selection import RFE
from sklearn.feature_selection import SelectKBest, f_regression

# Загрузка данных

In [2]:
%%capture
!wget https://www.dropbox.com/s/64ol9q9ssggz6f1/data_ford_price.xlsx

In [3]:
data = pd.read_excel('data_ford_price.xlsx') 

# работаем только с частью столбцов, выделю их
data = data[['price','year', 'cylinders', 'odometer', 'lat', 'long', 'weather']]
data.dropna(inplace = True)


## Метод рекурсивного исключения признаков

In [4]:
data_RFE = data.copy()

# подготовливаю выборки
y = data_RFE['price']
x = data_RFE.drop(columns='price')

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=40)

In [5]:
# выделяю 3 самых информативных признака
estimator = LinearRegression()
selector = RFE(estimator, n_features_to_select=3, step=1)
selector = selector.fit(X_train, y_train)

 
selector.get_feature_names_out()

array(['year', 'cylinders', 'lat'], dtype=object)

In [6]:
# исключаю признаки, которые не выбрала функция
X_train = X_train.drop(columns =['odometer', 'long', 'weather'])
X_test = X_test.drop(columns =['odometer', 'long', 'weather'])

X_train.head()

,year,cylinders,lat
4641,2011,6,31.316064
3188,2010,8,42.892121
135,2008,8,35.887189
962,2011,8,43.781240
6188,2006,8,42.484503


In [7]:
# обучаю модель
estimator.fit(X_train, y_train)
pred_test = estimator.predict(X_test)

# вывожу метрику
print('MAE на тестовой выборке',mean_absolute_error(y_test, pred_test).round())


MAE на тестовой выборке 5097.0


##  ВЫБОР ПРИЗНАКОВ НА ОСНОВЕ ФИЛЬТРОВ

In [8]:
data_SB = data.copy()

# подготовливаю выборки
y = data_SB['price']
x = data_SB.drop(columns='price')

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=40)

In [9]:
# выделяю 3 самых информативных признака

selector = SelectKBest(f_regression, k=3)
selector.fit(X_train, y_train)
 
selector.get_feature_names_out()

array(['year', 'cylinders', 'odometer'], dtype=object)

In [10]:
# исключаю признаки, коорые не выбрала функция
X_train =  X_train.drop(columns =['lat','long', 'weather'])
X_test = X_test.drop(columns =['lat','long', 'weather'])

X_train.head()

,year,cylinders,odometer
4641,2011,6,122882
3188,2010,8,144570
135,2008,8,136124
962,2011,8,136668
6188,2006,8,187000


In [11]:
# обучаю модель
estimator.fit(X_train, y_train)
pred_test2 = estimator.predict(X_test)

# вывожу метрику
print('MAE на тестовой выборке',mean_absolute_error(y_test, pred_test2).round())

MAE на тестовой выборке 4709.0


In [12]:
print('MAE на тестовой выборке, при методе на основе фильтров -',mean_absolute_error(y_test, pred_test2).round())
print('MAE на тестовой выборке, при методе RFE',mean_absolute_error(y_test, pred_test).round())

MAE на тестовой выборке, при методе на основе фильтров - 4709.0
MAE на тестовой выборке, при методе RFE 5097.0


Вывод.

В данном случае, метод ВЫБОРА ПРИЗНАКОВ НА ОСНОВЕ ФИЛЬТРОВ показал лучший результат метрики MAE